<a href="https://colab.research.google.com/github/beriaacan/Radon-Analysis-with-Deep-Learning-Algorithms/blob/main/1_a_Data_Collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#libraries
import pandas as pd
from googleapiclient.discovery import build
from google.oauth2 import service_account
import time
from prophet import Prophet
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import seaborn as sns #korelasyon matrisi icin
import matplotlib.pyplot as plt
import os

In [2]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
FOLDERNAME ='bitirme/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

In [4]:
# Constants
SERVICE_ACCOUNT_FILE = 'drive/MyDrive/bitirme/radondata-1234-aacc234f0c5a.json'
SPREADSHEET_ID = '1oddL5AANo3HQPJyKxSqiuOg7N7z7Zie9PtUmaBvO9ho'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

In [5]:
creds = None
creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

In [6]:
# Servisi oluştur
service = build("sheets", "v4", credentials=creds)

# Erişmek istediğiniz Google Sheets dosyasının ID'si
SPREADSHEET_ID = '1oddL5AANo3HQPJyKxSqiuOg7N7z7Zie9PtUmaBvO9ho'  # Google Sheets dosya ID'sini ekleyin

# Google Sheets dosyasındaki sayfa adlarını al
spreadsheet_info = service.spreadsheets().get(spreadsheetId=SPREADSHEET_ID).execute()
sheet_names = [sheet['properties']['title'] for sheet in spreadsheet_info['sheets']]

# Elde edilen sayfa adlarını yazdır
print("Sheet Names:", sheet_names)

Sheet Names: ['data_+905367791473_tel', 'data_+905396792291_tel', 'data_+905396792336_tel', 'data_+905396792349_tel', 'data_+905396792354_tel', 'data_+905396792367_tel', 'data_+905396792370_tel', 'data_+905396792372_tel', 'data_+905396792374_tel', 'data_+905396792381_tel', 'data_+905396792391_tel', 'data_+905396792392_tel', 'data_+905396792413_tel', 'data_+905396792421_tel', 'data_+905396792427_tel', 'data_+905396792434_tel', 'data_+905396792435_tel', 'data_+905396792437_tel', 'data_+905396792438_tel', 'data_+905396792448_tel', 'data_+905396792452_tel', 'data_+905396792459_tel', 'data_+905396792487_tel', 'data_+905396792489_tel', 'data_+905396792490_tel', 'data_+905396792491_tel', 'data_+905396792843_tel', 'data_+905396792846_tel', 'data_+905396792851_tel', 'data_+905396792925_tel', 'data_+905396793144_tel', 'data_+905396793146_tel', 'data_+905396793149_tel', 'data_+905396793157_tel', 'data_+905396793168_tel', 'data_+905396793171_tel', 'data_+905396793181_tel', 'data_+905396793193_tel'

In [7]:
# Specify the sheet name you want to access
chosen_sheet = 'data_+905367791473_tel'

result = service.spreadsheets().values().get(spreadsheetId=SPREADSHEET_ID, range=chosen_sheet).execute()

# Extract the values from the result
values = result.get('values', [])

# Check if there is any data
if not values:
    print('No data found.')
else:
    # Convert the data to a Pandas DataFrame
    df_chosen_sheet = pd.DataFrame(values[1:], columns=values[0])

    # Print the DataFrame
    print(f"Selected Sheet: {chosen_sheet}")

df_chosen_sheet.head()

Selected Sheet: data_+905367791473_tel


,tarih,amain,meanAM0,stDevG1,meanAM1,stDevG2,meanAM6,stDeva6,meanAM7,stDeva7,...,meanTemp,stDevT,meanHum,stDevH,meanPres,stDevP,Su_motor,stDev_motor,ML,ML_distance
0,22.8.23-15:48:9,"1,001","417,001","0,471","104,001","36,091","513,001","0,361","209,001","18,791",...,"42,251","0,441","18,541","3,081","1005,781","0,071","62,391","0,591",None,None
1,22.8.23-17:39:19,"1,001","417,001","0,371","67,001","0,471","513,001","0,421","236,001","16,481",...,"42,641","0,651","17,371","0,751","1005,541","0,041","64,421","0,321",None,None
2,22.8.23-18:40:31,"2,001","417,001","0,501","65,001","0,471","513,001","0,261","242,001","1,871",...,"38,731","1,121","26,781","3,741","1005,721","0,121","62,861","0,961",None,None
3,22.8.23-19:41:44,"3,001","415,001","0,471","64,001","0,901","513,001","0,361","245,001","0,541",...,"35,331","0,891","33,671","1,771","1006,471","0,191","56,581","1,631",None,None
4,22.8.23-20:42:56,"4,001","415,001","0,501","64,001","0,471","513,001","0,131","246,001","1,121",...,"32,511","0,651","40,511","2,201","1006,971","0,111","52,421","0,951",None,None


In [8]:
# Function to get sheet information
def get_sheet_info(service, spreadsheet_id, sheet_name):
    result = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=sheet_name).execute()
    values = result.get('values', [])
    num_rows = len(values)
    num_columns = len(values[0]) if values and len(values) > 0 else 0
    return {
        'sheet_name': sheet_name,
        'num_rows': num_rows,
        'num_columns': num_columns
    }

# Get information for all sheets
all_sheet_info = []
for sheet_name in sheet_names:
    sheet_info = get_sheet_info(service, SPREADSHEET_ID, sheet_name)
    all_sheet_info.append(sheet_info)

# Convert the information to a Pandas DataFrame
df_sheet_info = pd.DataFrame(all_sheet_info)

# Print the DataFrame
print('Sheet Information:')
print(df_sheet_info)

Sheet Information:
                sheet_name  num_rows  num_columns
0   data_+905367791473_tel      2925           22
1   data_+905396792291_tel      2977           22
2   data_+905396792336_tel      3214           22
3   data_+905396792349_tel      2997           22
4   data_+905396792354_tel      2679           22
5   data_+905396792367_tel      1306           22
6   data_+905396792370_tel      1375           22
7   data_+905396792372_tel         1           22
8   data_+905396792374_tel      2071           22
9   data_+905396792381_tel      2986           22
10  data_+905396792391_tel         1           22
11  data_+905396792392_tel      3006           22
12  data_+905396792413_tel         1           22
13  data_+905396792421_tel      2497           22
14  data_+905396792427_tel      3108           22
15  data_+905396792434_tel      3010           22
16  data_+905396792435_tel      2982           22
17  data_+905396792437_tel      2757           22
18  data_+905396792438_tel     

In [13]:
# Filter out rows where num_rows is 1
filtered_df_sheet_info = df_sheet_info[df_sheet_info['num_rows'] > 2000]
#ben burada küçültmüş oldum elimdeki verisetini ama eksik data coktu bu kısmı tekrar güncelleyebilirim

print('Filtered Sheet Information:')
print(filtered_df_sheet_info)

Filtered Sheet Information:
                sheet_name  num_rows  num_columns
0   data_+905367791473_tel      2925           22
1   data_+905396792291_tel      2977           22
2   data_+905396792336_tel      3214           22
3   data_+905396792349_tel      2997           22
4   data_+905396792354_tel      2679           22
8   data_+905396792374_tel      2071           22
9   data_+905396792381_tel      2986           22
11  data_+905396792392_tel      3006           22
13  data_+905396792421_tel      2497           22
14  data_+905396792427_tel      3108           22
15  data_+905396792434_tel      3010           22
16  data_+905396792435_tel      2982           22
17  data_+905396792437_tel      2757           22
19  data_+905396792448_tel      2894           22
22  data_+905396792487_tel      2892           22
23  data_+905396792489_tel      2884           22
24  data_+905396792490_tel      2071           22
25  data_+905396792491_tel      2961           22
26  data_+905396792843

In [14]:
len(filtered_df_sheet_info)

40

###COPY FILE###

In [15]:
# Create a directory to store CSV files
csv_folder_path = '/content/drive/My Drive/bitirme/radon_analysis_final/full_data_csv'
os.makedirs(csv_folder_path, exist_ok=True)

# Function to get sheet values for a specific sheet
def get_sheet_values(service, spreadsheet_id, sheet_name):
    result = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=sheet_name).execute()
    values = result.get('values', [])
    return values

# Save each sheet as a CSV file in the specified folder
for index, row in filtered_df_sheet_info.iterrows():
    sheet_name = row['sheet_name']
    sheet_values = get_sheet_values(service, SPREADSHEET_ID, sheet_name)

    # Convert the sheet values to a Pandas DataFrame
    df_sheet_content = pd.DataFrame(sheet_values[1:], columns=sheet_values[0])

    # Exclude the last two columns
    df_sheet_content = df_sheet_content.iloc[:, :-2]

    # Save the DataFrame as a CSV file
    csv_file_path = os.path.join(csv_folder_path, f"{sheet_name}.csv")
    df_sheet_content.to_csv(csv_file_path, index=False)

    print(f"CSV file saved: {csv_file_path}")

print(f'All CSV files saved to: {csv_folder_path}')


CSV file saved: /content/drive/My Drive/bitirme/radon_analysis_final/full_data_csv/data_+905367791473_tel.csv
CSV file saved: /content/drive/My Drive/bitirme/radon_analysis_final/full_data_csv/data_+905396792291_tel.csv
CSV file saved: /content/drive/My Drive/bitirme/radon_analysis_final/full_data_csv/data_+905396792336_tel.csv
CSV file saved: /content/drive/My Drive/bitirme/radon_analysis_final/full_data_csv/data_+905396792349_tel.csv
CSV file saved: /content/drive/My Drive/bitirme/radon_analysis_final/full_data_csv/data_+905396792354_tel.csv
CSV file saved: /content/drive/My Drive/bitirme/radon_analysis_final/full_data_csv/data_+905396792374_tel.csv
CSV file saved: /content/drive/My Drive/bitirme/radon_analysis_final/full_data_csv/data_+905396792381_tel.csv
CSV file saved: /content/drive/My Drive/bitirme/radon_analysis_final/full_data_csv/data_+905396792392_tel.csv
CSV file saved: /content/drive/My Drive/bitirme/radon_analysis_final/full_data_csv/data_+905396792421_tel.csv
CSV file s